## Version Information

<table align="left">
  <thead>
    <tr>
      <th>Component</th>
      <th>Version</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>python</td>
      <td>3.11.5</td>
    </tr>
    <tr>
      <td>pytket</td>
      <td>1.28.0</td>
    </tr>
  </tbody>
</table>

</table>


In [24]:
from pytket import Circuit
from qiskit.circuit.library import QFT
from pytket.extensions.qiskit import tk_to_qiskit, qiskit_to_tk, AerBackend
from pytket.circuit.display import render_circuit_jupyter
from sympy import isprime
import math
from pytket.backends.backendresult import BackendResult
from fractions import Fraction
import random

In [25]:
def find_all_as(N):
    ns=[]
    for n in range(2,N):
        if math.gcd(n,N)==1:
            ns.append(n)
    return ns

def validate_a(a):
    ns = find_all_as(N)
    if a not in ns:
        raise ValueError(f"'a' must be one of {ns}")
    return True

def get_nlen_mlen(N):       
    n_len = math.ceil(math.log2(N))
    m_len = 2 * n_len
    return n_len, m_len 

def is_power_of_prime(N):
    for base in range(2, int(math.sqrt(N)) + 1):
        power = 2
        while (result := base ** power) <= N:
            if result == N:
                return True
            power += 1
    return False

def validate_N(N):
    if isprime(N):
        raise ValueError(f"{N} is a prime.")
    if is_power_of_prime(N):
        raise ValueError(f"{N} is a power of a prime.")
    return True


def check_r_condition(a, r, N):
    if r % 2 == 0 and pow(a, r // 2, N) != N - 1:
        return True
    else:
        return False
    
    
# 100 of prime numbers
prime_100 = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 
             107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 
             227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 
             349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 
             467, 479, 487, 491, 499, 503, 509, 521, 523, 541]

def generate_N():
    ACCEPTED_N = False
    max_qubits = 29
    while not ACCEPTED_N:
        ps = random.sample(prime_100[5:], 2)
        N = ps[0]*ps[1]
        n_len, m_len = get_nlen_mlen(N)
        if n_len+m_len<=max_qubits:
            print(f"N = {ps[0]} * {ps[1]} = {N}")
            return N

In [52]:
def initiate_qc(N):
    # identify n-length and m-length
    n_len, m_len = get_nlen_mlen(N)
    
    qc = Circuit(m_len + n_len, m_len)
    
    # Initialize counting qubits
    # in state |+>
    for q in range(m_len):
        qc.H(q)

    # And auxiliary register in state |1>
    qc.X(m_len)

    return qc


def append_c_amodN(qc, a, N, barrier=False):
    n_len, m_len = get_nlen_mlen(N)
    
    for q in range(m_len):
        pwr = 2**q
        for i, bit in enumerate(format(pow(a, pwr, N) ^ 1, f'0{n_len}b')):
            if bit == '1':
                qc.CX(q, i + m_len)
        qc.add_barrier(qc.qubits) if barrier else _
    
    return qc



def append_IQFT(qc, N):
    n_len, m_len = get_nlen_mlen(N)
    
    qiskit_circuit = tk_to_qiskit(qc)
    iqft_circuit = QFT(m_len, inverse=True)
    qiskit_circuit.append(QFT(m_len, inverse=True), range(m_len))
    qc = qiskit_to_tk(qiskit_circuit)
    
    return qc



# this function just to print what inside controlled U gate
def print_c_amodN(a, N):
    n_len, m_len = get_nlen_mlen(N)
    
    for p in range(m_len):
        pwr = 2**p
        _txt = f'a={a}, N={N}, p={p}, 2^p=pwr={pwr}'
        print(f"\n{_txt}\n{len(_txt)*'-'}")
    
        print(f"{format(pow(a,pwr,N), f'0{n_len}b')} | {a}^{pwr} mod {N} = {pow(a,pwr,N)}")
        print(f"{format(pow(a,pwr,N)^1, f'0{n_len}b')} | x_gates")
        

def simulation(qc, shots=1000):
    backend = AerBackend()
    compiled_circuit = backend.get_compiled_circuit(qc)
    handle = backend.process_circuit(compiled_circuit, n_shots=shots)
    result = backend.get_result(handle)
    counts = result.get_counts()
    counts = [''.join(map(str, tpl))[::-1] for tpl in counts]

    return list(set(counts))


def shor_qc(a, N, display=False, shots=1):
    _, m_len = get_nlen_mlen(N)
    
    qc = initiate_qc(N)
    qc = append_c_amodN(qc, a, N)  
    qc = append_IQFT(qc, N)

    for i in range(m_len):
        qc.Measure(i, i)
    
    render_circuit_jupyter(qc) if display else _
    
    return simulation(qc, shots)

In [62]:
N=15
a=2
validate_N(N)
validate_a(a)

_, m_len = get_nlen_mlen(N)

# simple circuit 
qc = initiate_qc(N)
qc = append_c_amodN(qc, a, N)
qc = append_IQFT(qc, N)

for i in range(m_len):
    qc.Measure(i, i)

render_circuit_jupyter(qc)
print(simulation(qc))


# circuit with barrier
qc = initiate_qc(N)
qc = append_c_amodN(qc, a, N, barrier=True)
qc = append_IQFT(qc, N)

for i in range(m_len):
    qc.Measure(i, i)

render_circuit_jupyter(qc)
print(simulation(qc))

['00000000', '01000000', '11000000', '10000000']


['00000000', '01000000', '11000000', '10000000']


In [59]:
FACTOR_FOUND = False
ATTEMPT = 0

N = generate_N()


# validate_N(N)  # no need to validate N value

n_len, m_len = get_nlen_mlen(N)

while not FACTOR_FOUND:
    ATTEMPT = 0
    for a in range(2,N):
        if math.gcd(a,N)==1:
            ATTEMPT += 1
            _txt = f'\n[a = {a}]'
            print(f"{_txt}\n{'-'*len(_txt)}")
            print(f"ATTEMPT {ATTEMPT}:")
            
            readings = shor_qc(a, N)
            for reading in readings:
                print(f"\nRegister Reading: {reading} [{int(reading,2)}]")
                phase = int(reading,2)/(2**m_len)
                frac = Fraction(phase).limit_denominator(N)
                r = frac.denominator
                print(f"Corresponding Phase: [{int(reading,2)}/{(2**m_len)}] >>> {phase:0.5f} >>> {frac}")
                
                print(f"Result: r = {r}")

                if not check_r_condition(a, r, N):
                    print(f"[{r}] did not pass r conditions")
                    continue

                if phase != 0:
                    # Guesses for factors are gcd(x^{r/2} ±1 , 15)
                    guesses = [math.gcd(a**(r//2)-1, N), math.gcd(a**(r//2)+1, N)]
                    print(f"Guessed Factors: {guesses[0]} and {guesses[1]}")
                    for guess in guesses:
                        if guess not in [1,N] and (N % guess) == 0:
                            # Guess is a factor!
                            print(f"*** Non-trivial factor found: {guess} ***")
                            FACTOR_FOUND = True
                            
                if FACTOR_FOUND:
                    break

        else:
            print(f'\n# [a={a}] is not an accepted value! #')
            
        if FACTOR_FOUND:
            break
    
    if not FACTOR_FOUND:
        print('\nALL VALID A VALUES ARE USED !')
        FACTOR_FOUND = True

N = 31 * 13 = 403

[a = 2]
--------
ATTEMPT 1:
['010111011110001000']

Register Reading: 010111011110001000 [96136]
Corresponding Phase: [96136/262144] >>> 0.36673 >>> 139/379
Result: r = 379
[379] did not pass r conditions

[a = 3]
--------
ATTEMPT 2:
['101000011110101000']

Register Reading: 101000011110101000 [165800]
Corresponding Phase: [165800/262144] >>> 0.63248 >>> 74/117
Result: r = 117
[117] did not pass r conditions

[a = 4]
--------
ATTEMPT 3:
['001100010010110101']

Register Reading: 001100010010110101 [50357]
Corresponding Phase: [50357/262144] >>> 0.19210 >>> 34/177
Result: r = 177
[177] did not pass r conditions

[a = 5]
--------
ATTEMPT 4:
['100011000100001101']

Register Reading: 100011000100001101 [143629]
Corresponding Phase: [143629/262144] >>> 0.54790 >>> 183/334
Result: r = 334
Guessed Factors: 1 and 1

[a = 6]
--------
ATTEMPT 5:
['001000011001101010']

Register Reading: 001000011001101010 [34410]
Corresponding Phase: [34410/262144] >>> 0.13126 >>> 34/259
Result

In [60]:
print_c_amodN(a, N)


a=7, N=403, p=0, 2^p=pwr=1
--------------------------
000000111 | 7^1 mod 403 = 7
000000110 | x_gates

a=7, N=403, p=1, 2^p=pwr=2
--------------------------
000110001 | 7^2 mod 403 = 49
000110000 | x_gates

a=7, N=403, p=2, 2^p=pwr=4
--------------------------
110000010 | 7^4 mod 403 = 386
110000011 | x_gates

a=7, N=403, p=3, 2^p=pwr=8
--------------------------
100100001 | 7^8 mod 403 = 289
100100000 | x_gates

a=7, N=403, p=4, 2^p=pwr=16
---------------------------
001100100 | 7^16 mod 403 = 100
001100101 | x_gates

a=7, N=403, p=5, 2^p=pwr=32
---------------------------
101001000 | 7^32 mod 403 = 328
101001001 | x_gates

a=7, N=403, p=6, 2^p=pwr=64
---------------------------
110000010 | 7^64 mod 403 = 386
110000011 | x_gates

a=7, N=403, p=7, 2^p=pwr=128
----------------------------
100100001 | 7^128 mod 403 = 289
100100000 | x_gates

a=7, N=403, p=8, 2^p=pwr=256
----------------------------
001100100 | 7^256 mod 403 = 100
001100101 | x_gates

a=7, N=403, p=9, 2^p=pwr=512
-------